In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
long_reads = sort(filter(x -> occursin(r"\.fna\.badread", x) && !occursin("filtlong", x), readdir(genome_dir, join=true)), by=x->filesize(x))
fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && !occursin(".vcf", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))
# done through 3
# filtered_long_reads = filter(x -> occursin(fasta_files[6], x), long_reads)

In [ ]:
threads = 1
memory = 2
# need to run on 1000x from 
for in_fastq in long_reads
    out_fastq = replace(in_fastq, ".fq.gz" => ".filtlong.fq.gz")
    if !isfile(out_fastq)
        # --min_length 1000
        # removed from small genome simulation filterings
        cmd = 
        """
        $(Mycelia.MAMBA) run --live-stream -n filtlong filtlong --min_mean_q 10 --keep_percent 90 $(in_fastq) | gzip > $(out_fastq)
        """
        # Mycelia.scg_sbatch(
        #     job_name = "$(basename(out_fastq))",
        #     mail_user = "cameron.prybol@gmail.com",
        #     logdir = mkpath("$(homedir())/workspace/slurmlogs"),
        #     partition = "batch",
        #     account = "mpsnyder",
        #     mem_gb = memory,
        #     cpus_per_task= threads,
        #     cmd = cmd)
    Mycelia.nersc_sbatch(
        job_name = "$(basename(out_fastq))",
        mail_user = "cameron.prybol@gmail.com",
        logdir = mkpath("$(homedir())/workspace/slurmlogs"),
        qos = "shared",
        mem_gb = memory,
        cpus_per_task = threads,
        cmd = cmd)
    else
        @info "$(out_fastq) already present"
    end
end

In [ ]:
# nanoq 0.10.0

# Filters and summary reports for nanopore reads

# USAGE:
#     nanoq [FLAGS] [OPTIONS]

# FLAGS:
#     # -f, --fast       Ignore quality values if present
#     # -h, --help       Prints help information
#     # -H, --header     Header for summary output
#     # -j, --json       Summary report in JSON format
#     # -s, --stats      Summary report only [stdout]
#     # -V, --version    Prints version information
#     # -v, --verbose    Verbose output statistics [multiple, up to -vvv]

# OPTIONS:
#     # -c, --compress-level <1-9>     Compression level to use if compressing output [default: 6]
#     -i, --input <input>            Fast{a,q}.{gz,xz,bz}, stdin if not present
#     # -m, --max-len <INT>            Maximum read length filter (bp) [default: 0]
#     # -w, --max-qual <FLOAT>         Maximum average read quality filter (Q) [default: 0]
#     -l, --min-len <INT>            Minimum read length filter (bp) [default: 0]
#     -q, --min-qual <FLOAT>         Minimum average read quality filter (Q) [default: 0]
#     -o, --output <output>          Output filepath, stdout if not present
#     -O, --output-type <u|b|g|l>    u: uncompressed; b: Bzip2; g: Gzip; l: Lzma
#     -r, --report <FILE>            Summary read statistics report output file
#     -t, --top <INT>                Number of top reads in verbose summary [default: 5]
#     -L, --read-lengths <FILE>      Output read lengths of surviving reads to file
#     -Q, --read-qualities <FILE>    Output read qualities of surviving reads to file
#     -S, --trim-start <INT>         Trim bases from the start of each read [default: 0]
#     -E, --trim-end <INT>           Trim bases from the end of each read [default: 0]
# # MIT
# # https://github.com/esteinig/nanoq

# !mamba install -c conda-forge -c bioconda nanoq
# !nanoq \
# -vvv \
# --min-len 1000 \
# --min-qual 10 \
# --json \
# --input /content/drive/MyDrive/20231103.simulate-long-reads/GCF_000005845.2_ASM584v2_genomic.annotated.fna.50x.fastq.gz \
# --report /content/drive/MyDrive/20231103.simulate-long-reads/GCF_000005845.2_ASM584v2_genomic.annotated.fna.50x.fastq.nanoq.report.json \
# --output /content/drive/MyDrive/20231103.simulate-long-reads/GCF_000005845.2_ASM584v2_genomic.annotated.fna.50x.fastq.nanoq.fq.gz

# filtlong 